In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)


Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [5]:
# Subset the training data for faster turnaround.
train_subset = 10000
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation (with L2 Regularization).
  logits = tf.matmul(tf_train_dataset, weights) + biases
  reg = tf.nn.l2_loss(weights)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + beta * reg)

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 801

with tf.Session(graph=graph) as session:
  # One-time operation to initialize parameters.
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 45.873276
Training accuracy: 10.3%
Validation accuracy: 13.5%
Loss at step 100: 11.567609
Training accuracy: 74.1%
Validation accuracy: 72.2%
Loss at step 200: 4.381528
Training accuracy: 79.5%
Validation accuracy: 76.4%
Loss at step 300: 1.942692
Training accuracy: 82.4%
Validation accuracy: 79.5%
Loss at step 400: 1.115230
Training accuracy: 84.1%
Validation accuracy: 81.1%
Loss at step 500: 0.831210
Training accuracy: 84.6%
Validation accuracy: 81.7%
Loss at step 600: 0.732480
Training accuracy: 84.8%
Validation accuracy: 82.0%
Loss at step 700: 0.697813
Training accuracy: 84.9%
Validation accuracy: 82.1%
Loss at step 800: 0.685537
Training accuracy: 85.0%
Validation accuracy: 82.1%
Test accuracy: 88.9%


In [7]:
batch_size = 128
hidden_nodes_num = 1024
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes_num]))
  biases_1 = tf.Variable(tf.zeros([hidden_nodes_num]))
    
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
  
  weights_2 = tf.Variable(tf.truncated_normal([hidden_nodes_num, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation (with L2 Regularization).
  logits = tf.matmul(hidden, weights_2) + biases_2
  reg = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + beta * reg)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3442.214355
Minibatch accuracy: 8.6%
Validation accuracy: 28.3%
Minibatch loss at step 500: 21.212490
Minibatch accuracy: 88.3%
Validation accuracy: 84.3%
Minibatch loss at step 1000: 0.950898
Minibatch accuracy: 80.5%
Validation accuracy: 83.0%
Minibatch loss at step 1500: 0.582527
Minibatch accuracy: 87.5%
Validation accuracy: 83.7%
Minibatch loss at step 2000: 0.614991
Minibatch accuracy: 90.6%
Validation accuracy: 83.4%
Minibatch loss at step 2500: 0.714458
Minibatch accuracy: 83.6%
Validation accuracy: 83.3%
Minibatch loss at step 3000: 0.771784
Minibatch accuracy: 83.6%
Validation accuracy: 83.3%
Test accuracy: 90.3%


In [9]:
num_steps = 3001
dataset_size = 512

train_dataset_small = train_dataset[:dataset_size, :]
train_labels_small = train_labels[:dataset_size, :]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_small[offset:(offset + batch_size), :]
    batch_labels = train_labels_small[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3506.287109
Minibatch accuracy: 12.5%
Validation accuracy: 34.6%
Minibatch loss at step 500: 21.003227
Minibatch accuracy: 100.0%
Validation accuracy: 77.0%
Minibatch loss at step 1000: 0.452814
Minibatch accuracy: 100.0%
Validation accuracy: 78.5%
Minibatch loss at step 1500: 0.306546
Minibatch accuracy: 100.0%
Validation accuracy: 78.4%
Minibatch loss at step 2000: 0.262446
Minibatch accuracy: 100.0%
Validation accuracy: 78.4%
Minibatch loss at step 2500: 0.270870
Minibatch accuracy: 100.0%
Validation accuracy: 78.8%
Minibatch loss at step 3000: 0.283556
Minibatch accuracy: 100.0%
Validation accuracy: 78.6%
Test accuracy: 85.7%


In [10]:
batch_size = 128
hidden_nodes_num = 1024
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes_num]))
  biases_1 = tf.Variable(tf.zeros([hidden_nodes_num]))
    
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
  # Dropout.
  keep_prob = tf.placeholder(tf.float32)
  hidden_dropout = tf.nn.dropout(hidden, keep_prob)
    
  weights_2 = tf.Variable(tf.truncated_normal([hidden_nodes_num, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation (with L2 Regularization).
  logits = tf.matmul(hidden_dropout, weights_2) + biases_2
  reg = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + beta * reg)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

In [11]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3628.589355
Minibatch accuracy: 4.7%
Validation accuracy: 29.0%
Minibatch loss at step 500: 21.351892
Minibatch accuracy: 83.6%
Validation accuracy: 83.8%
Minibatch loss at step 1000: 1.046640
Minibatch accuracy: 76.6%
Validation accuracy: 83.0%
Minibatch loss at step 1500: 0.645756
Minibatch accuracy: 86.7%
Validation accuracy: 83.0%
Minibatch loss at step 2000: 0.666876
Minibatch accuracy: 92.2%
Validation accuracy: 83.3%
Minibatch loss at step 2500: 0.767900
Minibatch accuracy: 83.6%
Validation accuracy: 83.0%
Minibatch loss at step 3000: 0.821785
Minibatch accuracy: 82.0%
Validation accuracy: 83.0%
Test accuracy: 90.0%


In [12]:
num_steps = 3001
dataset_size = 512

train_dataset_small = train_dataset[:dataset_size, :]
train_labels_small = train_labels[:dataset_size, :]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_small[offset:(offset + batch_size), :]
    batch_labels = train_labels_small[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3649.524902
Minibatch accuracy: 8.6%
Validation accuracy: 32.1%
Minibatch loss at step 500: 21.107361
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Minibatch loss at step 1000: 0.485725
Minibatch accuracy: 100.0%
Validation accuracy: 78.7%
Minibatch loss at step 1500: 0.348416
Minibatch accuracy: 99.2%
Validation accuracy: 78.5%
Minibatch loss at step 2000: 0.284825
Minibatch accuracy: 100.0%
Validation accuracy: 78.5%
Minibatch loss at step 2500: 0.292581
Minibatch accuracy: 100.0%
Validation accuracy: 78.6%
Minibatch loss at step 3000: 0.321125
Minibatch accuracy: 100.0%
Validation accuracy: 78.7%
Test accuracy: 86.0%


In [13]:
batch_size = 128
hidden_nodes_num = 1024
beta = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes_num]))
  biases_1 = tf.Variable(tf.zeros([hidden_nodes_num]))
    
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
  # Dropout.
  keep_prob = tf.placeholder(tf.float32)
  hidden_dropout = tf.nn.dropout(hidden, keep_prob)
    
  weights_2 = tf.Variable(tf.truncated_normal([hidden_nodes_num, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation (with L2 Regularization).
  logits = tf.matmul(hidden_dropout, weights_2) + biases_2
  reg = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + beta * reg)
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

  # optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

In [14]:
num_steps = 10000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 734.933716
Minibatch accuracy: 12.5%
Validation accuracy: 29.3%
Minibatch loss at step 500: 213.444809
Minibatch accuracy: 75.0%
Validation accuracy: 79.7%
Minibatch loss at step 1000: 117.603447
Minibatch accuracy: 80.5%
Validation accuracy: 80.5%
Minibatch loss at step 1500: 69.739685
Minibatch accuracy: 82.0%
Validation accuracy: 81.5%
Minibatch loss at step 2000: 41.458347
Minibatch accuracy: 83.6%
Validation accuracy: 83.1%
Minibatch loss at step 2500: 25.270344
Minibatch accuracy: 84.4%
Validation accuracy: 84.1%
Minibatch loss at step 3000: 15.551856
Minibatch accuracy: 82.8%
Validation accuracy: 84.7%
Minibatch loss at step 3500: 9.701081
Minibatch accuracy: 80.5%
Validation accuracy: 85.7%
Minibatch loss at step 4000: 6.072481
Minibatch accuracy: 83.6%
Validation accuracy: 85.9%
Minibatch loss at step 4500: 3.838942
Minibatch accuracy: 85.2%
Validation accuracy: 86.0%
Minibatch loss at step 5000: 2.584683
Minibatch accuracy: 85.9%
Validati